In [1]:
import pandas as pd
import os
import gene_tools as gt


os.chdir("C:/Users/alber/OneDrive/Documents/GitHub/ProjetBioinfo")

print(os.getcwd())
os.listdir("data")

C:\Users\alber\OneDrive\Documents\GitHub\ProjetBioinfo


['expl_analysis.png', '00_BPStart']

In [2]:
from gene_tools import foldersLoad, cleanDic, compute_scores, geneScores, NA_filtering,evaluate_trait_scores,evaluate_OR
from Scoring.scomet import Mean, Max, Min, Median, pca

root_path = os.path.join('data', '00_BPStart/00_gene_prioritization')

folders = ['eQTL_GWAS_blood', 'Exome', 'GWAS','pQTL_GWAS']
raw_data = foldersLoad(root_path, folders)
merged = pd.read_csv(r"data\00_BPStart\merged_drug_all_db_with_hgnc_TTD.tsv", sep="\t")
dic_clean = gt.cleanDic(raw_data)


In [3]:
from Scoring.scomet import Mean, Max, Min, Median, pca, Product

ldl_clean = dic_clean["LDL"]

Max(ldl_clean)
Mean(ldl_clean)
Median(ldl_clean)
Min(ldl_clean)
Product(ldl_clean)

score_cols = ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"]
drug_targets = set(merged[merged['trait'] == 'LDL'].loc[merged[merged['trait'] == 'LDL']["Sum"] >= 3,"EnsemblId"])   
                                                                                                             
for col in score_cols:
    top = ldl_clean.loc[ldl_clean[col] < 0.5]
    top_ids = set(top["EnsemblId"])
    overlap = top_ids & drug_targets
    percent = 100 * len(overlap) / len(top_ids)
    print(f"{col}: {percent:.2f}% of top 1% are drug targets")
    
for col in score_cols:
    top = ldl_clean.sort_values(by=col, ascending=True).head(int(0.01 * len(ldl_clean)))
    top_ids = set(top["EnsemblId"])
    overlap = top_ids & drug_targets
    percent = 100 * len(overlap) / len(top_ids)
    print(f"{col}: {percent:.2f}% of top 1% are drug targets")


    
len(ldl_clean["GWAS_percentile"].dropna())

Prioscore_mean: 20.00% of top 1% are drug targets
Prioscore_max: 7.69% of top 1% are drug targets
Prioscore_min: 6.01% of top 1% are drug targets
Prioscore_median: 22.22% of top 1% are drug targets
Prioscore_product: 9.35% of top 1% are drug targets
Prioscore_mean: 4.41% of top 1% are drug targets
Prioscore_max: 2.94% of top 1% are drug targets
Prioscore_min: 6.37% of top 1% are drug targets
Prioscore_median: 4.90% of top 1% are drug targets
Prioscore_product: 5.39% of top 1% are drug targets


19197

In [4]:
evaluate_trait_scores(
    df = ldl_clean,
    drug_reference= merged,
    score_columns=["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median", "Prioscore_product"],
    overlap_method="topvalues",
    cutoff=0.01,
    printer=False,
    targetthreshold=3)

{'LDL': {'Prioscore_mean': 4.411764705882353,
  'Prioscore_max': 2.9411764705882355,
  'Prioscore_min': 6.372549019607843,
  'Prioscore_median': 4.901960784313726,
  'Prioscore_product': 5.392156862745098}}

In [5]:


from gene_tools.analysis import evaluate_OR

evaluate_OR(merged,ldl_clean,sum_threshold=3,cutoff = 0.01,score_cols = ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"],method="topvalues")


[LDL] Prioscore_mean: OR = 1.63, p = 1.3131e-01, drug targets in top 1% = 12
[LDL] Prioscore_max: OR = 1.06, p = 8.5133e-01, drug targets in top 1% = 8
[LDL] Prioscore_min: OR = 2.54, p = 6.3730e-04, drug targets in top 1% = 18
[LDL] Prioscore_median: OR = 1.93, p = 2.4393e-02, drug targets in top 1% = 14
[LDL] Prioscore_product: OR = 1.78, p = 5.8955e-02, drug targets in top 1% = 13


{'LDL': {'Prioscore_mean': [1.628, 0.131, 12],
  'Prioscore_max': [1.057, 0.851, 8],
  'Prioscore_min': [2.542, 0.001, 18],
  'Prioscore_median': [1.925, 0.024, 14],
  'Prioscore_product': [1.776, 0.059, 13]}}